In [1]:
### Import necessary libraries

from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_recommenders as tfrs

import os
import pprint
import tempfile

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
masterdf = pd.read_csv('../data_cleaned/brazildata_mod.csv')
masterdf.head(3)

,Unnamed: 0,order_id,order_purchase_timestamp,user_id,customer_city,customer_state,product_category,quantity,price,review_score,timestamp,product_code,product_id
0,0,e481f51cbdc54678b7cc49136f2d6af7,2017-10-02 10:56:33,9ef432eb6251297304e76186b10a928d,sao paulo,SP,housewares,1.0,29.99,4,1.506942e+09,87285b34884572647811a353c7ac498a,housewares SKU 0
1,1,53cdb2fc8bc7dce0b6741e2150273451,2018-07-24 20:41:37,b0830fb4747a6c6d20dea0b8c802d7ef,barreiras,BA,perfumery,1.0,118.70,4,1.532465e+09,595fac2a385ac33a80bd5114aec74eb8,perfumery SKU 0
2,2,47770eb9100c2d0c44946d9cf07ec65d,2018-08-08 08:38:49,41ce2a54c0b03bf3443c3d931a367089,vianopolis,GO,auto,1.0,159.90,5,1.533718e+09,aa4383b373c6aca5d8797843e5594415,auto SKU 0


In [3]:
### standardize item data types, especially string, float, and integer

masterdf[['user_id',      
          'product_id',  
         ]] = masterdf[['user_id','product_id']].astype(str)

# we will play around with the data type of the quantity, 
# which you shall see later it affects the accuracy of the prediction.

masterdf['quantity'] = masterdf['quantity'].astype(float)

In [4]:
interactions_dict = masterdf.groupby(['user_id', 'product_id', 'timestamp'])[ 'quantity'].sum().reset_index()

interactions_dict = {name: np.array(value) for name, value in interactions_dict.items()}
interactions = tf.data.Dataset.from_tensor_slices(interactions_dict)

items_dict = masterdf[['product_id']].drop_duplicates()
items_dict = {name: np.array(value) for name, value in items_dict.items()}
items = tf.data.Dataset.from_tensor_slices(items_dict)

interactions = interactions.map(lambda x: {
                                            'user_id' : x['user_id'], 
                                            'product_id' : x['product_id'], 
                                            'quantity' : float(x['quantity']),
                                            "timestamp": x["timestamp"] })

items = items.map(lambda x: x['product_id'])

In [ ]:
## Basic housekeeping to prepare feature vocabularies

## timestamp is an exmaple of continuous features, which needs to be rescaled, or otherwise it will be 
## too large for the model.
## there are other methods to reduce the size of the timestamp, ,such as standardization and normalization
## here we use discretization, which puts them into buckets of categorical features, 

timestamps = np.concatenate(list(interactions.map(lambda x: x["timestamp"]).batch(100)))
max_timestamp = timestamps.max()
min_timestamp = timestamps.min()
timestamp_buckets = np.linspace(
    min_timestamp, max_timestamp, num=1000,)

item_titles = interactions.batch(10_000).map(lambda x: x["product_id"])
user_ids = interactions.batch(10_000).map(lambda x: x["user_id"])

unique_item_titles = np.unique(np.concatenate(list(item_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

In [6]:
tf.random.set_seed(42)
shuffled = interactions.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(60_000)
test = shuffled.skip(60_000).take(20_000)

cached_train = train.shuffle(100_000).batch(2048)
cached_test = test.batch(4096).cache()

# Model implementation

We split the query and candidate model separately to allow more stacked embedding layers before we pass it into the model.

In the user model (query model), in addition to user embedding, we also add timestamp embedding.

In [7]:
### user model

class UserModel(tf.keras.Model):

    def __init__(self, use_timestamps):
        super().__init__()

        self._use_timestamps = use_timestamps

        ## embed user id from unique_user_ids
        self.user_embedding = tf.keras.Sequential([
            tf.keras.layers.experimental.preprocessing.StringLookup(
                vocabulary=unique_user_ids, mask_token=None),
            tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),
        ])

        ## embed timestamp
        if use_timestamps:
            self.timestamp_embedding = tf.keras.Sequential([
              tf.keras.layers.experimental.preprocessing.Discretization(timestamp_buckets.tolist()),
              tf.keras.layers.Embedding(len(timestamp_buckets) + 1, 32),
            ])
            self.normalized_timestamp = tf.keras.layers.experimental.preprocessing.Normalization()

            self.normalized_timestamp.adapt(timestamps)

    def call(self, inputs):
        if not self._use_timestamps:
              return self.user_embedding(inputs["user_id"])

        ## all features here
        return tf.concat([
            self.user_embedding(inputs["user_id"]),
            self.timestamp_embedding(inputs["timestamp"]),
            self.normalized_timestamp(inputs["timestamp"]),
        ], axis=1)

For the candidate model, we want the model to learn from the text features too by processing the text features that is able to learn words that are similar to each other. It can also identify OOV (out of Vocabulary) word, so if we are predicing a new item, the model can calculate them appropriately.

Below, the item name will be transformated by tokenization (splitting into constituent words or word-pieces), followed by vocabulary learning, then followed by an embedding.

In [ ]:
### candidate model

class ItemModel(tf.keras.Model):

    def __init__(self):
        super().__init__()

        max_tokens = 10_000

        ## embed title from unique_item_titles
        self.title_embedding = tf.keras.Sequential([
          tf.keras.layers.experimental.preprocessing.StringLookup(
              vocabulary=unique_item_titles, mask_token=None),
          tf.keras.layers.Embedding(len(unique_item_titles) + 1, 32)
        ])

        ## processing text features: item title vectorizer (see self.title_vectorizer)
        self.title_vectorizer = tf.keras.layers.experimental.preprocessing.TextVectorization(
            max_tokens=max_tokens)

        ## we apply title vectorizer to items
        self.title_text_embedding = tf.keras.Sequential([
          self.title_vectorizer,
          tf.keras.layers.Embedding(max_tokens, 32, mask_zero=True),
          tf.keras.layers.GlobalAveragePooling1D(),
        ])

        self.title_vectorizer.adapt(items)

    def call(self, titles):
        return tf.concat([
            self.title_embedding(titles),
            self.title_text_embedding(titles),
        ], axis=1)

With both UserModel and ItemModel defined, we can put together a combined model and implement our loss and metrics logic.

Note that we also need to make sure that the query model and candidate model output embeddings of compatible size. Because we'll be varying their sizes by adding more features, the easiest way to accomplish this is to use a dense projection layer after each model:

In [9]:
class RetailModel(tfrs.models.Model):

    def __init__(self, use_timestamps):
        super().__init__()
        
        ## query model is user model
        self.query_model = tf.keras.Sequential([
          UserModel(use_timestamps),
          tf.keras.layers.Dense(32)
        ])
        
        ## candidate model is the item model
        self.candidate_model = tf.keras.Sequential([
          ItemModel(),
          tf.keras.layers.Dense(32)
        ])
        
        ## retrieval task, choose metrics
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=items.batch(128).map(self.candidate_model),
            ),
        )

    def compute_loss(self, features, training=False):
        # We only pass the user id and timestamp features into the query model. This
        # is to ensure that the training inputs would have the same keys as the
        # query inputs. Otherwise the discrepancy in input structure would cause an
        # error when loading the query model after saving it.
        
        query_embeddings = self.query_model({
            "user_id": features["user_id"],
            "timestamp": features["timestamp"],
        })
        
        item_embeddings = self.candidate_model(features["product_id"])

        return self.task(query_embeddings, item_embeddings)

Baseline: no timestamp features

We're ready to try out our first model: let's start with not using timestamp features to establish our baseline.

In [10]:
model = RetailModel(use_timestamps=False)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
model.fit(cached_train, epochs=3)
model.evaluate(cached_test, return_dict=True)

Epoch 1/3
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
30/30 [==============================] - 15s 401ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0032 - factorized_top_k/top_5_categorical_accuracy: 0.0100 - factorized_top_k/top_10_categorical_accuracy: 0.0139 - factorized_top_k/top_50_categorical_accuracy: 0.0284 - factorized_top_k/top_100_categorical_accuracy: 0.0378 - loss: 14858.1789 - regularization_loss: 0.0000e+00 - total_loss: 14858.1789
Epoch 2/3
30/30 [==============================] - 13s 407ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0217 - factorized_top_k/top_5_categorical_accuracy: 0.0758 - factorized_top_k/top_10_categorical_accuracy: 0.1067 - factorized_top_k/top_50_categorical_accuracy: 0.2140 - factorized_top_k/top_100_categorical_accuracy: 0.2801 - loss: 12659.2235 - regularization_loss: 0.0000e+00 - total_loss: 12659.2235
Epoch 3/3
30/30 [==============================] - 14s

Consider rewriting this model with the Functional API.
4/4 [==============================] - 4s 615ms/step - factorized_top_k/top_1_categorical_accuracy: 2.3104e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0023 - factorized_top_k/top_10_categorical_accuracy: 0.0035 - factorized_top_k/top_50_categorical_accuracy: 0.0097 - factorized_top_k/top_100_categorical_accuracy: 0.0166 - loss: 24718.6561 - regularization_loss: 0.0000e+00 - total_loss: 24718.6561


{'factorized_top_k/top_1_categorical_accuracy': 0.00023103581042960286,
 'factorized_top_k/top_5_categorical_accuracy': 0.0023103582207113504,
 'factorized_top_k/top_10_categorical_accuracy': 0.003465537214651704,
 'factorized_top_k/top_50_categorical_accuracy': 0.009703503921627998,
 'factorized_top_k/top_100_categorical_accuracy': 0.016557566821575165,
 'loss': 5160.34521484375,
 'regularization_loss': 0,
 'total_loss': 5160.34521484375}

Including time into the model:

Do the result change if we add time features?

In [12]:
model =  RetailModel(use_timestamps=True)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

model.fit(cached_train, epochs=3)

model.evaluate(cached_test, return_dict=True)

Epoch 1/3
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
30/30 [==============================] - 17s 471ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0083 - factorized_top_k/top_5_categorical_accuracy: 0.0133 - factorized_top_k/top_10_categorical_accuracy: 0.0174 - factorized_top_k/top_50_categorical_accuracy: 0.0361 - factorized_top_k/top_100_categorical_accuracy: 0.0544 - loss: 14498.4631 - regularization_loss: 0.0000e+00 - total_loss: 14498.4631
Epoch 2/3
30/30 [==============================] - 15s 476ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0158 - factorized_top_k/top_5_categorical_accuracy: 0.0511 - factorized_top_k/top_10_categorical_accuracy: 0.0774 - factorized_top_k/top_50_categorical_accuracy: 0.1855 - factorized_top_k/top_100_categorical_accuracy: 0.2624 - loss: 12165.9211 - regularization_loss: 0.0000e+00 - total_loss: 12165.9211
Epoch 3/3
30/30 [==============================] - 15s

{'factorized_top_k/top_1_categorical_accuracy': 0.000616095494478941,
 'factorized_top_k/top_5_categorical_accuracy': 0.003850596956908703,
 'factorized_top_k/top_10_categorical_accuracy': 0.006546014454215765,
 'factorized_top_k/top_50_categorical_accuracy': 0.02140931785106659,
 'factorized_top_k/top_100_categorical_accuracy': 0.03596457466483116,
 'loss': 5379.62255859375,
 'regularization_loss': 0,
 'total_loss': 5379.62255859375}

Eventhough we only run it at three epochs, we can see accuracy increase as we add time into the model.